In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

#read the datasets
# train_df = pd.read_csv("./train.csv")
# val_df = pd.read_csv("./val.csv")
# test_df = pd.read_csv("./test.csv")
df = pd.read_csv("./df2.csv")
df = df.rename(columns={"Unnamed: 0": "label_id"})
text = df.text.values
labels = df.label_id.values
# train_df=train_df.rename(columns={"Unnamed: 0": "label_id"})
# val_df=val_df.rename(columns={"Unnamed: 0": "label_id"})
# test_df=test_df.rename(columns={"Unnamed: 0": "label_id"})
# train_text = train_df.text.values
# train_labels = train_df.label_id.values
# val_text = val_df.text.values
# val_labels = val_df.label_id.values
# test_text = test_df.text.values
# test_labels = test_df.label_id.values
# train_labels = train_labels.astype(float)
'''for idx in range(len(train_labels)):
    train_labels[idx] = train_labels[idx].astype('float32')
print(type(train_labels))
print(type(train_labels[0]))'''

#tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

#helper function to check tokenization of sentences
def print_rand_sentence():
  idx = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[idx]), tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[idx]))]).T
  print(tabulate(table, headers = ['Tokens', 'Token IDs'], tablefmt = 'fancy_grid'))

# print_rand_sentence()

#tokenize the corpus
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(input_text,add_special_tokens = True,  max_length = 64,padding = 'max_length',
                        return_attention_mask = True,return_tensors = 'pt',truncation=True)

for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)
# print(token_id[196])

# helper function to check encoding of sentences
def print_rand_sentence_encoding():
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]
  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, headers = ['Tokens', 'Token IDs', 'Attention Mask'],tablefmt = 'fancy_grid'))

# print_rand_sentence_encoding()

#create dataset
ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16
train_i, val_i = train_test_split(np.arange(len(labels)),test_size = ratio,shuffle = True)
# Train and validation sets
# train = TensorDataset(token_id[train_i], attention_masks[train_i], labels[train_i])
train = TensorDataset(token_id[:], attention_masks[:], labels[:])
val = TensorDataset(token_id[val_i], attention_masks[val_i], labels[val_i])
# Prepare DataLoader
train_dataloader = DataLoader(train,sampler = RandomSampler(train),batch_size = batch_size)
validation_dataloader = DataLoader(val,sampler = SequentialSampler(val),batch_size = batch_size)

#Main Training
#Define eval metrics
def b_tp(preds, labels):
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])
def b_fp(preds, labels):
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])
def b_tn(preds, labels):
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])
def b_fn(preds, labels):
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])
def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
            'bert-base-uncased',num_labels = len(labels),output_attentions = False,output_hidden_states = False,)
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5,eps = 1e-08)


#Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# print(device)
epochs = 16
for _ in trange(epochs, desc = 'Epoch'):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask, labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    model.eval() 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')



In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf
from tabulate import tabulate
from tqdm import trange
import random
import tqdm

In [2]:
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv("./df2.csv")
df = df.rename(columns={"Unnamed: 0": "label_id"})

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [5]:
text = df.text.values
labels = df.category.values

In [6]:
label_to_id = {}
id_to_label = {}
j = 0
for i in np.unique(labels):
  label_to_id[i] = j
  id_to_label[j] = i
  j += 1

In [7]:
label_to_id

{'African daisy': 0,
 'African violet': 1,
 'Dahlia': 2,
 'Douglas-fir': 3,
 'English walnut': 4,
 'Jerusalem cherry': 5,
 'Persian violet': 6,
 'alfalfa': 7,
 'almond': 8,
 'anemone': 9,
 'animal': 10,
 'apple': 11,
 'apricot': 12,
 'asparagus': 13,
 'avocado': 14,
 'azalea': 15,
 'banana and plantain': 16,
 'barley': 17,
 'beet': 18,
 'bellflower': 19,
 'black walnut': 20,
 'bleeding heart': 21,
 'butterfly flower': 22,
 'cacao': 23,
 'caneberries': 24,
 'canola': 25,
 'carnation': 26,
 'carrot': 27,
 'cassava': 28,
 'cattleya': 29,
 'chickpea': 30,
 'cineraria': 31,
 'citrus': 32,
 'coconut palm': 33,
 'coffee': 34,
 'common bean': 35,
 'cotton': 36,
 'crucifer': 37,
 'cucurbit': 38,
 'cyclamen': 39,
 'date palm': 40,
 'durian': 41,
 'elm': 42,
 'flax': 43,
 'foliage plant': 44,
 'fuchsia': 45,
 'geranium': 46,
 'grape': 47,
 'hazelnut': 48,
 'hemp': 49,
 'holiday cacti': 50,
 'hop': 51,
 'hydrangea': 52,
 'impatiens': 53,
 'kalanchoe': 54,
 'lentil': 55,
 'lettuce': 56,
 'lisianthu

In [8]:
id_to_label

{0: 'African daisy',
 1: 'African violet',
 2: 'Dahlia',
 3: 'Douglas-fir',
 4: 'English walnut',
 5: 'Jerusalem cherry',
 6: 'Persian violet',
 7: 'alfalfa',
 8: 'almond',
 9: 'anemone',
 10: 'animal',
 11: 'apple',
 12: 'apricot',
 13: 'asparagus',
 14: 'avocado',
 15: 'azalea',
 16: 'banana and plantain',
 17: 'barley',
 18: 'beet',
 19: 'bellflower',
 20: 'black walnut',
 21: 'bleeding heart',
 22: 'butterfly flower',
 23: 'cacao',
 24: 'caneberries',
 25: 'canola',
 26: 'carnation',
 27: 'carrot',
 28: 'cassava',
 29: 'cattleya',
 30: 'chickpea',
 31: 'cineraria',
 32: 'citrus',
 33: 'coconut palm',
 34: 'coffee',
 35: 'common bean',
 36: 'cotton',
 37: 'crucifer',
 38: 'cucurbit',
 39: 'cyclamen',
 40: 'date palm',
 41: 'durian',
 42: 'elm',
 43: 'flax',
 44: 'foliage plant',
 45: 'fuchsia',
 46: 'geranium',
 47: 'grape',
 48: 'hazelnut',
 49: 'hemp',
 50: 'holiday cacti',
 51: 'hop',
 52: 'hydrangea',
 53: 'impatiens',
 54: 'kalanchoe',
 55: 'lentil',
 56: 'lettuce',
 57: 'lisia

In [9]:
label_id = []
for i in labels:
  label_id.append(label_to_id[i])

In [10]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [11]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in enumerate(df['text']):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [12]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

In [13]:
labels2 = np.zeros((len(df), len(set(label_id))))

In [14]:
labels2[np.arange(len(df)), label_id] = 1

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels2))
dataset.take(196)

<_TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(101,), dtype=tf.float64, name=None))>

In [16]:
def TCDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [17]:
dataset = dataset.map(TCDatasetMapFunction)

In [18]:
dataset.take(196)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(101,), dtype=tf.float64, name=None))>

In [19]:
dataset = dataset.shuffle(2000).batch(16, drop_remainder=True)

In [20]:
p = 0.8
train_size = int((len(df)//16)*p)

In [21]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [22]:
from transformers import TFBertModel

In [23]:
model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [30]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

tc_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [31]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [32]:
tc_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [33]:
hist = tc_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Epoch 1/32
97/97 [==============================] - 145s 1s/step - loss: 4.6952 - accuracy: 0.0129 - val_loss: 4.5530 - val_accuracy: 0.0350
Epoch 2/32
97/97 [==============================] - 95s 981ms/step - loss: 4.5254 - accuracy: 0.0387 - val_loss: 4.4934 - val_accuracy: 0.0250
Epoch 3/32
97/97 [==============================] - 95s 975ms/step - loss: 4.4717 - accuracy: 0.0387 - val_loss: 4.4382 - val_accuracy: 0.0350
Epoch 4/32
97/97 [==============================] - 97s 997ms/step - loss: 4.4529 - accuracy: 0.0322 - val_loss: 4.4295 - val_accuracy: 0.0400
Epoch 5/32
97/97 [==============================] - 95s 978ms/step - loss: 4.4349 - accuracy: 0.0374 - val_loss: 4.4393 - val_accuracy: 0.0300
Epoch 6/32
97/97 [==============================] - 95s 980ms/step - loss: 4.3945 - accuracy: 0.0451 - val_loss: 4.3518 - val_accuracy: 0.0550
Epoch 7/32
97/97 [==============================] - 94s 974ms/step - loss: 4.3982 - accuracy: 0.0522 - val_loss: 4.3240 - val_accuracy: 0.0750
E

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer2')(intermediate_layer)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer2) # softmax -> calcs probs of classes

tc_model2 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model2.summary()
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model2.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
intermediate_layer2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer2')(intermediate_layer)
intermediate_layer3 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer3')(intermediate_layer2)
output_layer = tf.keras.layers.Dense(len(set(label_id)), activation='softmax', name='output_layer')(intermediate_layer3) # softmax -> calcs probs of classes

tc_model3 = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
tc_model3.summary()
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
tc_model3.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = tc_model3.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=32
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           